# Reading an NWB File
After downloading an NWB file, you *may* want to view the data inside. You can do basic reading of the file with **[PyNWB](https://github.com/NeurodataWithoutBorders/pynwb)**. This is a package designed to utilize, modify, and process NWB files. The basic read functionality of PyNWB is shown below.

### Environment Setup
⚠️**Note: If running on a new environment, run this cell once and then restart the kernel**⚠️

In [1]:
import warnings
warnings.filterwarnings('ignore')

try:
    from databook_utils.dandi_utils import dandi_stream_open
except:
    !git clone https://github.com/AllenInstitute/openscope_databook.git
    %cd openscope_databook
    %pip install -e .
    %cd ./docs/basics

In [2]:
import pandas as pd

from dandi import dandiapi
from pynwb import NWBHDF5IO

### Downloading an NWB File
To read an NWB File, it must first be downloaded. `dandiset_id` and `dandi_filepath` may be changed to select a different file off of DANDI. If the file of interest is already downloaded, you don't need to run the download cell again. When trying to download an embargoed file, refer to the code from the [Downloading an NWB File](./download_nwb.ipynb) notebook.

In [3]:
dandiset_id = "000021"
dandi_filepath = "sub-723627600/sub-723627600_ses-742951821.nwb"
download_loc = "."

In [4]:
filename = dandi_filepath.split("/")[-1]
filepath = f"{download_loc}/{filename}"

print(filename)
print(filepath)

sub-723627600_ses-742951821.nwb
./sub-723627600_ses-742951821.nwb


In [5]:
client = dandiapi.DandiAPIClient()
my_dandiset = client.get_dandiset(dandiset_id)
file = my_dandiset.get_asset_by_path(dandi_filepath)
# this may take awhile, especially if the file to download is large
file.download(filepath)

print(f"Downloaded file to {filepath}")

A newer version (0.62.2) of dandi/dandi-cli is available. You are using 0.61.2


Downloaded file to ./sub-723627600_ses-742951821.nwb


### Reading an NWB File
You can read in a NWB file with **PyNWB**'s `NWBHDF5IO` method to retrieve an io object. You can use the `.read` method to actually read it in. From there, you can see the raw data of the NWB file and can print the fields you are interested in. 

In [11]:
# takes relative or absolute filepath as first argument
io = NWBHDF5IO(f"{download_loc}/{filename}", mode="r", load_namespaces=True)
nwb = io.read()
nwb

,center_x,center_y,height,phi,width,timestamps
id,,,,,,
0,327.339200,220.528876,7.122461,-0.046604,8.540818,3.15554
1,327.225841,220.183568,7.286199,-0.186530,8.163864,3.17849
2,327.171088,220.285960,7.290239,-0.267194,8.074097,3.18553
3,327.097132,220.502671,7.239620,-0.006504,8.026307,3.21892
,center_x,center_y,height,phi,width,timestamps
id,,,,,,
0,310.748365,207.967287,124.601937,0.080571,158.078800,3.15554
1,311.049063,208.004817,124.397462,0.078970,157.795426,3.17849
2,311.266888,208.144224,124.609645,0.089244,157.970495,3.18553


In [7]:
### uncomment these to view aspects of the file

### not all of these exist for all NWB files (Key Errors will arise if the fields don't exist for this file)

# nwb.identifier
# nwb.processing
# nwb.acquisition["events"]
# nwb.intervals["trials"]
# nwb.stimulus["StimulusPresentation"]
# nwb.electrodes

### OpenScope's File Schemas
OpenScope works with two main types of brain data files for different experiments. There's Extracellular Electrophysiology, *Ecephys*, which uses **Neuropixels** probes to gather electrical data, and Optical Physiology, *Ophys*, which uses **2-Photon** Calcium imaging to gather fluorescence data. Since the NWB Standard is very flexible, and these files are used for different needs, there are several differences between these data types. These table schemas were made as comprehensive descriptions of what goes in our NWB files. Below, **Pandas** is used to display a general outline for each file type where every field within the NWB file is described. Note that NWB files or files from other institutions could vary from these significantly. These tables are just meant to be used as a reference, don't be overwhelmed trying to understand all the fields!

In [8]:
# change pandas settings to show whole table
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

#### Ecephys File

In [9]:
pd.read_csv("../../data/ecephys_schema.csv")

,Section,Field,Type,Path,Definition
0,NaN,Session Description,string,nwb.session_description,Description of the Session
1,NaN,Session Identifier,string,nwb.identifier,Unique identifier of the session
2,NaN,Session Start Time,datetime,nwb.session_start_time,Datetime start of the session
3,NaN,Reference Time,datetime,nwb.timestamps_reference_time,Time that all timestamps are in reference to
4,NaN,Experiment Description,string,nwb.experiment_description,Description of the experiment
5,NaN,Institution,string,nwb.institution,Name of the institution that produced these data
6,acquisition,Corneal Reflection Tracking Comments,string,"nwb.acquisition[""EyeTracking""].corneal_reflection_tracking.comments",Comments
7,acquisition,Corneal Reflection Tracking Conversion,number,"nwb.acquisition[""EyeTracking""].corneal_reflection_tracking.conversion",Scalar to multiply each element in data to convert it to the specified unit
8,acquisition,Corneal Reflection Tracking Description,string,"nwb.acquisition[""EyeTracking""].corneal_reflection_tracking.description",Description
9,acquisition,Corneal Reflection Tracking Interval,number,"nwb.acquisition[""EyeTracking""].corneal_reflection_tracking.interval",Unexplained


#### Ophys File

In [10]:
pd.read_csv("../../data/ophys_schema.csv")

,Section,Field,Type,Path,Definition,Unnamed: 5
0,NaN,Session Description,string,nwb.session_description,Description of the Session,NaN
1,NaN,Session Identifier,string,nwb.identifier,Unique identifier of the session,NaN
2,NaN,Session Start Time,datetime,nwb.session_start_time,Datetime start of the session,NaN
3,NaN,Reference Time,datetime,nwb.timestamps_reference_time,Time that all timestamps are in reference to,NaN
4,NaN,Experiment Description,string,nwb.experiment_description,Description of the experiment,NaN
5,NaN,Institution,string,nwb.institution,Name of the institution that produced these data,NaN
6,acquisition,Corneal Reflection Tracking Comments,string,"nwb.acquisition[EyeTracking""].corneal_reflection_tracking.comments""",Comments,NaN
7,acquisition,Corneal Reflection Tracking Conversion,number,"nwb.acquisition[EyeTracking""].corneal_reflection_tracking.conversion""",Scalar to multiply each element in data to convert it to the specified unit,NaN
8,acquisition,Corneal Reflection Tracking Description,string,"nwb.acquisition[EyeTracking""].corneal_reflection_tracking.description""",Description,NaN
9,acquisition,Corneal Reflection Tracking Interval,number,"nwb.acquisition[EyeTracking""].corneal_reflection_tracking.interval""",Unexplained,NaN
